# Dive !

## Part 1 problem statement

(Adapted from [Advent of Code 2021, day 2](https://adventofcode.com/2021/day/2))

You will be given a series of instructions like

```txt
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

These instructions will change your horizontal position and your depth, two values you need to keep track of:

 - `forward X` increases the horizontal position by X units;
 - `down X` increases the depth by X units; and
 - `up X` decreases the depth by X units.

Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

 - `forward 5` adds 5 to your horizontal position, a total of 5.
 - `down 5` adds 5 to your depth, resulting in a value of 5.
 - `forward 8` adds 8 to your horizontal position, a total of 13.
 - `up 3` decreases your depth by 3, resulting in a value of 2.
 - `down 8` adds 8 to your depth, resulting in a value of 10.
 - `forward 2` adds 2 to your horizontal position, a total of 15.

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

**Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?**

_Using the input file `input.txt`, the result should be 1727835._

In [2]:
INPUT_FILE = 'input.txt'

In [3]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:
    command, value = line.split()
    value = int(value)

    if command == "forward":
        horiz_pos += value
    elif command == "up":
        depth -= value
    elif command == "down":
        depth += value
    else:
        raise ValueError('Unknown command.')

print(horiz_pos * depth)
    


1727835


There is nothing too wild going on here.

Perhaps the thing that can easily go most unnoticed is the fact that the line `command, value = line.split()` is already doing some input validation for us:
the fact that we are unpacking into `command, line` means we are assuming that `line.split()` returns two values.
If it returns any other number of values, we get a `ValueError`:

In [4]:
command, value = "cmd val otherthing".split()

ValueError: too many values to unpack (expected 2)

### Pattern matching

If you are using Python 3.10 or newer, you might be tempted to use [structural pattern matching](https://mathspp.com/blog/pydonts/structural-pattern-matching-tutorial) here.
We can write a solution using `match` that is remarkably similar to the solution using `if`:

In [6]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:
    command, value = line.split()
    value = int(value)
 
    match command:
        case "forward":
            horiz_pos += value
        case "up":
            depth -= value
        case "down":
            depth += value
        case _:
            raise ValueError("Unknown Command")


print(horiz_pos * depth)

1727835


So, is this any better?
We can argue it is _not_, because we didn't simplify our code, and yet managed to increase its depth.

To walk towards a scenario where pattern matching would be probably be more useful, let's rewrite the `match` statement:

In [7]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

horiz_pos, depth = 0, 0
for line in instructions:

    match line.split():
        case ["forward", value]:
            horiz_pos += int(value)
        case ["up", value]:
            depth -= int(value)
        case ["down", value]:
            depth += int(value)
        case _:
            raise ValueError("Unknown Command.")

print(horiz_pos * depth)


1727835


By matching directly the `line.split()` expression, we are making it easier for ourselves to handle instructions that have a different _structure_.
For example, imagine there was a `"reset"` instruction, that resetted the horizontal position and the depth to 0.
Using `match`, this is what the solution could look like:

In [8]:
with open(INPUT_FILE, 'r') as f:
    instructions = f.readlines()

instructions.append("reset")
horiz_pos, depth = 0, 0
for line in instructions:
    match line.split():
        case ["reset"]:
            horiz_pos, depth = 0, 0
        case ["forward", value]:
            horiz_pos += int(value)
        case["up", value]:
            depth -= int(value)
        case["down", value]:
            depth += int(value)
        case _:
            raise ValueError("Unkown Command.")
print(horiz_pos * depth)

0


We only needed to add two lines of code to handle this new command, and the handling of all commands looks similar: a `case` statement and some code.
If we were to do the same thing in the original `if` statement, we would have to special-case the `"reset"` command because we would have to check for it before unpacking the line into the `command` and `line` variables:

In [9]:
with open(INPUT_FILE, "r") as f:
    instructions = f.readlines()
    
instructions.append("reset")  # Add a "reset" command to the end.

horiz_pos, depth = 0, 0
for line in instructions:
    if line == "reset":
        horiz_pos, depth = 0, 0
        continue
    
    command, value = line.split()
    value = int(value)
    
    if command == "forward":
        horiz_pos += value
    elif command == "up":
        depth -= value
    elif command == "down":
        depth += value
    else:
        raise ValueError("Unknown command.")

print(horiz_pos * depth)  # Prints 0 because the last command was "reset".

0


So, in conclusion, for such a homogeneous set of commands, the `if` statement is preferable.
If the line structure were more heterogeneous, then structural pattern matching would start to show its benefits.

### How to end the `if` block

In the solution above, our `if` block compares `command` explicitly to each of the three possible commands, and uses the `else` to raise an error in the event that we receive a command we don't know.
We could have written, just as easily, the following `if` block:

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
else:
    depth += value
```

This block assumes that the variable `command` _always_ contains one of the three known commands, and thus uses the `else` to handle the `down` command.

However, there is a disadvantage to writing code like this:
one cannot look at the `if` block and _read_ what is the third case.
Is it a single one?
Are there multiple commands that map to the action of doing `depth += value`?

Thus, one can argue it is preferable to be explicit about the cases we are handling.
Of course, we can still choose to write the `if` block like so:

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
elif command == "down":
    depth += value
```

The difference, here, is that we do not include the `else` branch with the `raise` statement.
This says explicitly the commands that we are handling, while also showing that we do not expect to have to handle anything else.

Another slight variant would be to write

```py
if command == "forward":
    horiz_pos += value
elif command == "up":
    depth -= value
elif command == "down":
    depth += value
else:
    pass
```

This variant can be understood to mean “we assume something else might come through in the variable `command`, but we don't care about it”.

These are just minor variations of one another, and _your_ interpretation might not necessarily align with mine, but I find it to be an interesting exercise to think about the different ways in which similar pieces of code are read and understood.

---

As far as this problem is concerned, there isn't much we can do to improve our solution significantly.
The problem is straightforward enough that any attempts to be clever would do more harm than good.

Therefore, we will now cover the second part of the problem.
Then, because this is a fairly simple problem, it acts as a good toy example to introduce a couple of interesting tools that could be relevant for similar tasks, but that would represent too much overhead here.

## Part 2 problem statement

(Adapted from [Advent of Code 2021, day 2](https://adventofcode.com/2021/day/2))

Part 2 is a modification of part 1.
Now, not only do we have to keep track of the horizontal position and depth, we also have to keep track of the **aim**.
On top of that, the **same commands** now have a **different meaning**:


 - `down X` increases your aim by X units;
 - `up X` decreases your aim by X units; and
 - `forward X` does two things:
    - it increases your horizontal position by X units; and
    - it increases your depth by your aim multiplied by X.
    
Recall the previous example:

```txt
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

Now, the final result is different:

 - `forward 5` adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
 - `down 5` adds 5 to your aim, resulting in a value of 5.
 - `forward 8` adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
 - `up 3` decreases your aim by 3, resulting in a value of 2.
 - `down 8` adds 8 to your aim, resulting in a value of 10.
 - `forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, **calculate the horizontal position and depth** you would have after following the planned course.
**What do you get if you multiply your final horizontal position by your final depth?**

_Using the input file `input.txt`, the answer should be 1544000595._

### Modifying the baseline solution

In order to solve this new version of the problem, we just have to adapt the handling of each command: